In [179]:
from ecdsa import SECP256k1, SigningKey,VerifyingKey
import hashlib
import os

In [180]:
def generate_valid_key_pair():
    private_key = SigningKey.generate(curve=SECP256k1)
    public_key = private_key.get_verifying_key()
    return private_key.to_string().hex(), public_key.to_string().hex()

# Generate a valid pair of keys for testing
spending_private_key_hex, spending_pub_key_hex = generate_valid_key_pair()
viewing_private_key_hex, viewing_pub_key_hex = generate_valid_key_pair()

# Use these valid keys for testing
stealth_meta_address = (spending_pub_key_hex, viewing_pub_key_hex)

#this is whats posted to ENS registery and requirement for key generation.
print(spending_pub_key_hex)

804eb3b596e01d55a3e8b929e7d9b0143b173ba56e98f19cd1493f794d00b8af0780515184d4df737b03db7823116f03011ed170c458e4a967ac4b49ff516c27


In [181]:
def generate_stealth_address(stealth_meta_address):
    # Generate a random 32-byte entropy ephemeral private key
    ephemeral_private_key = SigningKey.generate(curve=SECP256k1)
    ephemeral_public_key = ephemeral_private_key.get_verifying_key()

    # Parse the spending and viewing public keys from the stealth meta-address
    spending_pub_key, viewing_pub_key = parse_stealth_meta_address(stealth_meta_address)

    # Compute the shared secret
    shared_secret = ephemeral_private_key.privkey.secret_multiplier * viewing_pub_key.pubkey.point
    hashed_shared_secret = hashlib.sha256(shared_secret.x().to_bytes(32, 'big')).digest()

    # Extract the view tag
    view_tag = hashed_shared_secret[0]

    # Compute recipient's stealth public key
    stealth_pub_key = SECP256k1.generator * int.from_bytes(hashed_shared_secret, 'big')

    # Compute recipient's stealth address (using a simple hash for demonstration)
    stealth_address = hashlib.sha256(stealth_pub_key.x().to_bytes(32, 'big')).hexdigest()

    return stealth_address, ephemeral_public_key.to_string('compressed'), view_tag

def parse_stealth_meta_address(stealth_meta_address):
    # Assuming stealth_meta_address is a tuple of (spending_pub_key, viewing_pub_key)
    spending_pub_key = load_public_key(stealth_meta_address[0])
    viewing_pub_key = load_public_key(stealth_meta_address[1])
    return spending_pub_key, viewing_pub_key

def load_public_key(key_hex):
    return VerifyingKey.from_string(bytes.fromhex(key_hex), curve=SECP256k1)

# Example usage

stealth_address1, ephemeral_pub_key1, view_tag = generate_stealth_address(stealth_meta_address)

In [182]:
print(stealth_address1)
print(ephemeral_pub_key1)
print(view_tag)

a9679c18ed97e6fc8960b7072d532be0c44ebed968c6fdb1d359bdb06b266b11
b'\x02\xa2\xce\x93\xa2\xd6\xee\xef\xd2_\xf3\xdcm\xf3\xf0D(\x84\x87\x9a\xa3\x9e&\xab\x9e\xfeL\x99\x97\xa3\x9f~S'
191


In [183]:
def check_stealth_address(ephemeral_pub_key_bytes, recipient_viewing_private_key_hex, view_tag):
    # Load the ephemeral public key
    ephemeral_pub_key = VerifyingKey.from_string(ephemeral_pub_key_bytes, curve=SECP256k1)

    # Load the recipient's viewing private key
    recipient_viewing_private_key = SigningKey.from_string(bytes.fromhex(recipient_viewing_private_key_hex), curve=SECP256k1)

    # Compute the shared secret
    shared_secret = recipient_viewing_private_key.privkey.secret_multiplier * ephemeral_pub_key.pubkey.point

    # Hash the shared secret
    hashed_shared_secret = hashlib.sha256(shared_secret.x().to_bytes(32, 'big')).digest()

    # Check the view tag (optional)
    calculated_view_tag = hashed_shared_secret[0]
    if calculated_view_tag != view_tag:
        return False  # The transaction is not for this recipient

    # Compute recipient's stealth public key
    stealth_pub_key = SECP256k1.generator * int.from_bytes(hashed_shared_secret, 'big')

    # Compute recipient's stealth address (using a simple hash for demonstration)
    stealth_address = hashlib.sha256(stealth_pub_key.x().to_bytes(32, 'big')).hexdigest()

    return stealth_address

# Example usage

stealth_address = check_stealth_address(ephemeral_pub_key1, viewing_private_key_hex, view_tag)
print(stealth_address)

a9679c18ed97e6fc8960b7072d532be0c44ebed968c6fdb1d359bdb06b266b11


In [239]:
import binascii

def generate_stealth_private_key(viewing_private_key_hex, spending_private_key_hex, ephemeral_pub_key_bytes):
    # Convert the viewing private key from hex and create a signing key object
    viewing_private_key = SigningKey.from_string(bytes.fromhex(viewing_private_key_hex), curve=SECP256k1)
    
    # Convert the ephemeral public key from bytes and create a verifying key object
    ephemeral_pub_key = VerifyingKey.from_string(ephemeral_pub_key_bytes, curve=SECP256k1)

    # Shared secret computation
    shared_secret_point = viewing_private_key.privkey.secret_multiplier * ephemeral_pub_key.pubkey.point

    # Hash the shared secret correctly
    shared_secret_hash = hashlib.sha256(shared_secret_point.x().to_bytes(32, 'big')).digest()

    # Add the hashed shared secret to the spending private key
    spending_private_key = SigningKey.from_string(bytes.fromhex(spending_private_key_hex), curve=SECP256k1)
    spending_key_int = int.from_bytes(spending_private_key.to_string(), 'big')
    shared_secret_int = int.from_bytes(shared_secret_hash, 'big')

    stealth_private_key_int = (spending_key_int + shared_secret_int) % SECP256k1.order

    # Return the stealth private key in hex format
    stealth_private_key = SigningKey.from_secret_exponent(stealth_private_key_int, curve=SECP256k1)
    return stealth_private_key.to_string().hex()


private_key_stealth = generate_stealth_private_key(viewing_private_key_hex, spending_private_key_hex, ephemeral_pub_key1)
# Convert the private key to bytes
private_key_bytes = binascii.unhexlify(private_key_stealth)

# Create a SigningKey object from the private key
sk = SigningKey.from_string(private_key_bytes, curve=SECP256k1)

# Get the corresponding VerifyingKey (public key)
vk = sk.get_verifying_key()

# Convert the public key to hex
public_key_hex = vk.to_string('compressed').hex()

print("Private Key:", private_key_stealth)
print("Public Key:", public_key_hex)


Private Key: 44b0717353d1dce0ce1220d5e90730fc963781a89463f70a473a90eab3d10fd8
Public Key: 0280858081eb2bd0e75c40a8a2b6a397abb7d7b14f09e5982407485327ef25a815


In [134]:
# Generate stealth private key
private_key_stealth_hex = generate_stealth_private_key(viewing_private_key_hex, spending_private_key_hex, ephemeral_pub_key1)

# Convert hex to SigningKey object
private_key_stealth = SigningKey.from_string(bytes.fromhex(private_key_stealth_hex), curve=SECP256k1)

# Derive the corresponding VerifyingKey (public key) object
stealth_public_key = private_key_stealth.get_verifying_key()

# Get the compressed form of the stealth public key
stealth_public_key_compressed_hex = stealth_public_key.to_string('compressed').hex()

# Now extract the x-coordinate from the compressed public key (skipping the prefix byte)
x_coord = stealth_public_key_compressed_hex[2:]

# Hash the x-coordinate and compare with the stealth address
hashed_x_coord = hashlib.sha256(bytes.fromhex(x_coord)).hexdigest()

print(f"Stealth public key (compressed): {stealth_public_key_compressed_hex}")
print(f"Hashed x-coordinate: {hashed_x_coord}")
print(f"Stealth address from check_stealth_address: {stealth_address}")

# Compare the hashed x-coordinate to the stealth address
if hashed_x_coord == stealth_address:
    print("The stealth public key corresponds to the stealth address.")
else:
    print("There is a mismatch between the stealth public key and the stealth address.")

Stealth public key (compressed): 03c6276b3062e057f2a8daefc09d3a1447641d29cf5e75268f713e17b039c1a658
Hashed x-coordinate: 7cfbbb01554da106ed050aea0631a949d9316ad84b56877532dccba819ab9129
Stealth address from check_stealth_address: 3df0d8be70412e3d2850d6ea41beb0ddfb24c34c42c5cc8acfcc1432f79a21ca
There is a mismatch between the stealth public key and the stealth address.


In [227]:
from ecdsa import SECP256k1, SigningKey, VerifyingKey
import hashlib

def generate_valid_key_pair():
    private_key = SigningKey.generate(curve=SECP256k1)
    public_key = private_key.get_verifying_key()
    return private_key.to_string().hex(), public_key.to_string('compressed').hex()

def parse_stealth_meta_address(stealth_meta_address):
    spending_pub_key = VerifyingKey.from_string(bytes.fromhex(stealth_meta_address[0]), curve=SECP256k1)
    viewing_pub_key = VerifyingKey.from_string(bytes.fromhex(stealth_meta_address[1]), curve=SECP256k1)
    print(viewing_pub_key.to_string('compressed').hex())
    print(spending_pub_key.to_string('compressed').hex())
    return spending_pub_key, viewing_pub_key

def generate_stealth_address(stealth_meta_address):
    ephemeral_private_key = SigningKey.generate(curve=SECP256k1)
    ephemeral_public_key = ephemeral_private_key.get_verifying_key()

    spending_pub_key, viewing_pub_key = parse_stealth_meta_address(stealth_meta_address)

    shared_secret = ephemeral_private_key.privkey.secret_multiplier * viewing_pub_key.pubkey.point
    hashed_shared_secret = hashlib.sha256(shared_secret.x().to_bytes(32, 'big')).digest()

    view_tag = hashed_shared_secret[0]

    stealth_pub_key = SECP256k1.generator * int.from_bytes(hashed_shared_secret, 'big')
   
    stealth_address = hashlib.sha256(stealth_pub_key.x().to_bytes(32, 'big')).hexdigest()

    return stealth_address, ephemeral_public_key.to_string('compressed').hex(), view_tag

def generate_stealth_private_key(viewing_private_key_hex, spending_private_key_hex, ephemeral_pub_key_hex):
    viewing_private_key = SigningKey.from_string(bytes.fromhex(viewing_private_key_hex), curve=SECP256k1)
    ephemeral_pub_key = VerifyingKey.from_string(bytes.fromhex(ephemeral_pub_key_hex), curve=SECP256k1)

    shared_secret_point = viewing_private_key.privkey.secret_multiplier * ephemeral_pub_key.pubkey.point
    shared_secret_hash = hashlib.sha256(shared_secret_point.x().to_bytes(32, 'big')).digest()

    spending_private_key = SigningKey.from_string(bytes.fromhex(spending_private_key_hex), curve=SECP256k1)
    spending_key_int = int.from_bytes(spending_private_key.to_string(), 'big')
    shared_secret_int = int.from_bytes(shared_secret_hash, 'big')

    stealth_private_key_int = (spending_key_int + shared_secret_int) % SECP256k1.order
    stealth_private_key = SigningKey.from_secret_exponent(stealth_private_key_int, curve=SECP256k1)

    stealth_public_key = stealth_private_key.get_verifying_key().to_string('compressed').hex()

    return stealth_private_key.to_string().hex(), stealth_public_key

# Example usage
spending_private_key_hex, spending_pub_key_hex = generate_valid_key_pair()
viewing_private_key_hex, viewing_pub_key_hex = generate_valid_key_pair()
stealth_meta_address = (spending_pub_key_hex, viewing_pub_key_hex)

stealth_address, ephemeral_pub_key_hex, view_tag = generate_stealth_address(stealth_meta_address)

stealth_private_key_hex, stealth_public_key_hex = generate_stealth_private_key(viewing_private_key_hex, spending_private_key_hex, ephemeral_pub_key_hex)

print("Stealth Address:", stealth_address)
print("Stealth Private Key:", stealth_private_key_hex)
print("Stealth Public Key:", stealth_public_key_hex)

0282e23d92ec96cbed76f3780f59acaa7013dc575ae1aeef919c5d98de2891caab
0391a27d2b98e3f15756c5219fb16ff0ebcf84f646c91cd22682a1db5409554c1a
Stealth Address: debdacf23a3506076cff55a977d9a3a2f52ea054bc42c4aa0c741630b775abd6
Stealth Private Key: 77c846c235e0de8784c2958fb21816874016090a3f03de9100d1d129759f764b
Stealth Public Key: 021d3e87ca803e314699070f4798f103097cd362699a3bc69bb8e6617dcd1f9ca0


In [191]:
import binascii

# Convert the private key to bytes
private_key_bytes = binascii.unhexlify(stealth_private_key_hex)

# Create a SigningKey object from the private key
sk = SigningKey.from_string(private_key_bytes, curve=SECP256k1)

# Get the corresponding VerifyingKey (public key)
vk = sk.get_verifying_key()

# Convert the public key to hex
public_key_hex = vk.to_string('compressed').hex()

print("Private Key:", stealth_private_key_hex)
print("Public Key:", public_key_hex)

Private Key: 573ed8d32cce9d50384616b895e85a5722c41fb63d32bc629d892f07135f7451
Public Key: 031d2ec4286c220f3a69c57066c95d4996535090a649a37b4869eef43d09efff46
